In [1]:
import dvc.api
import pandas as pd
import sys 
sys.path.append('../../')
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [167]:
from pytorch_lightning.utilities.deepspeed import convert_zero_checkpoint_to_fp32_state_dict
save_path = '/workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_2/epoch=4-step=8094.ckpt'
output_path = f'{save_path}/pytorch_model.bin'


In [168]:
convert_zero_checkpoint_to_fp32_state_dict(save_path, output_path)

Processing zero checkpoint '/workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_2/epoch=4-step=8094.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 6
Parsing checkpoint created by deepspeed==0.7.3
Reconstructed fp32 state dict with 283 params 582401281 elements
Saving fp32 state dict to /workspaces/multitask-llm-rnd/modelling/models/product_attribute_extraction_text_generation/version_2/epoch=4-step=8094.ckpt/pytorch_model.bin


In [169]:
model = LLM_MultitaskMultimodal.load_from_checkpoint(output_path)

INFO:root:Unused kwargs when getting google/mt5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_singlemodal_wishtitledesp2attrkvpair': None}, 'head_dict': {}}


In [170]:
def format_input(question, title, description, category):
    template = f'[question start] {question} [question end] [title start] {title} [title end] [description start] {description} [description end] [taxonomy start] {category} [taxonomy end]'
    task_prefix = 'Denoise attribute value given attribute key questions for product with description and taxonomy: '
    return task_prefix + template

In [171]:
# df = pd.read_json(dvc.api.get_url( 
#     'datasets/data/wish_attr_extract_label/processed2/appen_020323_030323_delivered_030623_validated_product_attr_textonly_train_t5denoiseformat.json', 
#     repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
# ), lines=True)

In [172]:
rec = df.sample(1).to_dict('records')[0]

In [180]:
input_text = format_input(rec['attr_name_value_pairs_all_lower_t5_denoise_question'], rec['title'], rec['description'], rec['category'])

In [174]:
# input_text = format_input('gender: <extra_id_0>', rec['title'], rec['description'], rec['category'])

In [190]:
input_text = format_input('sport or activity type: <extra_id_0>', rec['title'], rec['description'], rec['category'])

In [191]:
rec

{'label_ordering': 18723,
 'sample_method': 'only_text',
 'pid': '602b8cab460914324f037c51',
 'category': 'Sports > Sneakers > Skateboarding Shoes',
 'title': 'Spring Autumn 2021 Men Canvas Shoes High Top Sneakers Men Vulcanize Shoes Casual Lace-Up Breathable Plus Size 35-44 High Quality 202101@#148',
 'description': 'Brand Name:\nUpper Material:Canvas\nOrigin:CN(Origin)\nInsole Material:EVA\nFashion Element:Sewing\nPattern Type:Patchwork\nLining Material:Canvas\nSeason:Spring/Autumn\nClosure Type:Lace-Up\nHeel Height:Low (1cm-3cm)\nFit:Fits true to size, take your normal size 202101@#148',
 'main_img_url': nan,
 'rater_output_processed': 'Sports > Sneakers > Skateboarding Shoes > Sport or Activity Type > Skateboarding\nSports > Sneakers > Skateboarding Shoes > Materials > Ethylene Vinyl Acetate (EVA)\nSports > Sneakers > Skateboarding Shoes > Materials > Canvas',
 'attr_name_value_pairs_normalized': [['Materials', 'Canvas'],
  ['Materials', 'Ethylene Vinyl Acetate (EVA)'],
  ['Sport o

In [192]:
print(input_text)

Denoise attribute value given attribute key questions for product with description and taxonomy: [question start] sport or activity type: <extra_id_0> [question end] [title start] Spring Autumn 2021 Men Canvas Shoes High Top Sneakers Men Vulcanize Shoes Casual Lace-Up Breathable Plus Size 35-44 High Quality 202101@#148 [title end] [description start] Brand Name:
Upper Material:Canvas
Origin:CN(Origin)
Insole Material:EVA
Fashion Element:Sewing
Pattern Type:Patchwork
Lining Material:Canvas
Season:Spring/Autumn
Closure Type:Lace-Up
Heel Height:Low (1cm-3cm)
Fit:Fits true to size, take your normal size 202101@#148 [description end] [taxonomy start] Sports > Sneakers > Skateboarding Shoes [taxonomy end]


In [193]:
inputs = model.tokenizer(input_text, return_tensors='pt')

In [194]:
model.eval()
model.tokenizer.batch_decode(model.transformer.generate(**inputs))

['<pad> <extra_id_0>skateboarding <extra_id_1></s>']